# TCP/UDP 協議與安全測試

傳輸層兩大協議在安全測試中的核心概念和掃描技術。

## 1. TCP vs UDP 核心差異

| 特性 | TCP | UDP |
|------|-----|-----|
| **連接** | 面向連接 | 無連接 |
| **可靠性** | 可靠 | 不可靠 |
| **速度** | 慢 | 快 |
| **掃描特點** | 狀態明確 | 難以判斷 |
| **代表服務** | HTTP(80), SSH(22) | DNS(53), SNMP(161) |

## 2. TCP 旗標與連接

### TCP 旗標 (掃描核心)
| 旗標 | 掃描應用 |
|------|----------|
| **SYN** | SYN 掃描 (主要) |
| **ACK** | ACK 掃描 (防火牆檢測) |
| **FIN** | FIN 掃描 (隱蔽) |
| **RST** | 連接拒絕 |

### TCP 連接流程圖解

#### 三次握手 (Connection Establishment)
```
客戶端                        伺服器
   |                             |
   | 1. SYN (seq=100)           |
   |----------------------------->| 請求建立連接
   |                             |
   | 2. SYN+ACK (seq=300, ack=101) |
   |<-----------------------------| 確認並回應
   |                             |
   | 3. ACK (ack=301)           |
   |----------------------------->| 連接建立完成
   |                             |
   |        ESTABLISHED          |
```

#### 四次揮手 (Connection Termination)  
```
主動關閉方                    被動關閉方
     |                           |
     | 1. FIN+ACK (seq=500)     |
     |--------------------------->| 請求終止連接
     |                           |
     | 2. ACK (ack=501)         |
     |<---------------------------| 確認終止請求
     |                           |
     | 3. FIN+ACK (seq=800)     |
     |<---------------------------| 也請求終止
     |                           |
     | 4. ACK (ack=801)         |
     |--------------------------->| 連接完全關閉
     |                           |
     |         CLOSED            |
```

#### 檔案傳輸完整流程
```
時間軸: 建立 → 傳輸 → 終止

1. 三次握手建立連接
2. 資料傳輸 (PSH+ACK ↔ ACK)
3. 四次揮手終止連接

狀態變化:
客戶端: CLOSED → SYN_SENT → ESTABLISHED → FIN_WAIT → TIME_WAIT → CLOSED
伺服器: LISTEN → SYN_RCVD → ESTABLISHED → CLOSE_WAIT → LAST_ACK → CLOSED
```

**安全測試重點**: 
- **SYN 掃描**: 只執行步驟1-2，不完成連接
- **狀態分析**: 監控異常的連接狀態
- **會話劫持**: 預測序列號進行攻擊

## 3. UDP 傳輸流程圖解

### UDP 特性
- **無連接**: 無需握手，直接傳送
- **無狀態**: 不追蹤連接狀態  
- **快速**: 8 bytes 標頭，開銷小
- **不可靠**: 不保證送達，無重傳

### UDP 傳輸流程
```
客戶端                        伺服器
   |                             |
   | UDP 封包 (直接發送資料)      |
   |----------------------------->| 
   |                             |
   | UDP 回應 (可選)              |
   |<-----------------------------| 
   |                             |
   |        無連接狀態            |
```

### UDP 掃描流程圖解
```
掃描器                        目標主機
   |                             |
   | UDP 封包 → 開放連接埠       |
   |----------------------------->| (通常無回應)
   |                             |
   | UDP 封包 → 關閉連接埠       |
   |----------------------------->|
   |                             |
   | ICMP Port Unreachable       |
   |<-----------------------------| (明確表示關閉)
```

**掃描判斷邏輯**:
- 無回應 → 可能開放或被過濾
- ICMP 錯誤 → 連接埠關閉  
- 應用層回應 → 連接埠開放

**常見 UDP 服務回應**:
- DNS (53): 查詢回應
- SNMP (161): 管理資訊
- NTP (123): 時間同步

## 5. 連接埠狀態 & 實務應用

### 5.1 狀態判斷
| 狀態 | TCP 表現 | UDP 表現 |
|------|----------|----------|
| **開放** | SYN+ACK 回應 | 應用層回應或無回應 |
| **關閉** | RST 回應 | ICMP Port Unreachable |
| **過濾** | 無回應 | 無回應 |

### 5.2 掃描策略
1. **快速探索**: `nmap -sS --top-ports 1000`
2. **UDP 服務**: `nmap -sU -p 53,161,123`  
3. **綜合掃描**: `nmap -sS -sU -sV target`
4. **隱蔽模式**: `nmap -sF -T1 target`

### 5.3 安全注意事項
- **授權**: 確保獲得測試許可
- **速度**: 控制掃描速度避免觸發 IDS  
- **範圍**: 只掃描必要的目標和連接埠

In [ ]:
# TCP/UDP 掃描實務範例
echo "=== TCP 掃描範例 ==="

echo "1. SYN 掃描 (推薦)"
# nmap -sS -p 22,80,443 target

echo "2. Connect 掃描 (無 root 權限)"
# nmap -sT -p 22,80,443 target

echo "3. ACK 掃描 (防火牆檢測)"
# nmap -sA -p 22,80,443 target

echo ""
echo "=== UDP 掃描範例 ==="

echo "1. 基本 UDP 掃描"
# nmap -sU target

echo "2. 常用 UDP 服務掃描"
# nmap -sU -p 53,67,161,123,514 target

echo "3. UDP 版本偵測"
# nmap -sU -sV --top-ports 100 target

echo ""
echo "=== 組合掃描範例 ==="

echo "1. TCP + UDP 綜合掃描"
# nmap -sS -sU -sV target

echo "2. 快速全面掃描"
# nmap -sS --top-ports 1000 -sU --top-ports 100 target

echo "3. 隱蔽掃描"
# nmap -sF -sU -T1 -f target

echo ""
echo "=== 監控網路流量 ==="
echo "TCP 封包: tcpdump -i any tcp port 80"
echo "UDP 封包: tcpdump -i any udp port 53"  
echo "混合流量: tcpdump -i any '(tcp port 80) or (udp port 53)'"

echo ""
echo "=== 安全測試重點 ==="
echo "- TCP: 連接狀態分析、會話劫持檢測"
echo "- UDP: 服務發現、放大攻擊檢測"
echo "- 組合使用獲得完整的服務映射"